In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}")

✅ Gemini API key setup complete.


In [5]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

In [6]:
outline_agent = Agent(
    name = 'OutlineAgent',
    model = 'gemini-2.5-flash-lite',
    instruction = '''Create a blod outline for the given topic with:
    1. A catchy headlie
    2. An instruction hook
    3. 3-2 main sections with 2-3 buller points for each
    4. A concluding thought''',
    output_key="blog_outline",
)
print("✅ outline_agent created.")

✅ outline_agent created.


In [7]:
writer_agent = Agent(
    name="WriterAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    output_key="blog_draft", 
)

print("✅ writer_agent created.")

✅ writer_agent created.


In [8]:
editor_agent = Agent(
    name="EditorAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog",
)

print("✅ editor_agent created.")

✅ editor_agent created.


In [9]:
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


In [10]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Write a blog post about nepal")


 ### Created new session: debug_session_id

User > Write a blog post about nepal
OutlineAgent > ## Headline: Nepal: Beyond the Peaks – A Journey of the Soul

**Instruction Hook:** Dreaming of soaring mountains and ancient temples? While Nepal's majestic Himalayas are undeniably its crown jewel, this incredible country offers so much more. Prepare to have your breath taken away by a tapestry of vibrant cultures, spiritual awakenings, and unforgettable adventures that extend far beyond the Everest summit.

**Main Section 1: The Majestic Himalayas: More Than Just a View**

*   **Trekking Paradise:** From the iconic Annapurna Circuit to challenging Everest Base Camp treks, Nepal offers unparalleled opportunities for hikers of all levels. Experience breathtaking vistas, interact with remote villages, and push your own boundaries amidst some of the most stunning scenery on Earth.
*   **Spiritual Sanctuary:** Beyond the physical challenges, the Himalayas hold deep spiritual significance. Dis